In [10]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from time import time, sleep
import os
import datetime
import pyperclip
import json
import shutil
import path
import re
from bs4 import BeautifulSoup
import pymysql

In [ ]:
def get_date_numdays(begin, numdays):
    ''' 
    begin : datetime.datetime(2020, 5, 23)
    n_days : 30
    '''
    begin = datetime.datetime.today()
    
    date_li = [begin - datetime.timedelta(days=x) for x in range(numdays)]
    return date_li

def get_date_range(begin, end):
    ''' 
    begin : datetime.datetime(2020, 5, 23)
    end : datetime.datetime(2020, 3, 23)
    '''
    timedelta = begin - end
    date_li = [begin - datetime.timedelta(days=x) for x in range(timedelta.days)]
    return date_li


In [11]:
cafe24_db = pymysql.connect(
    user='csp', 
    passwd='!', 
    host='5 ', 
    db='statics', 
    charset='utf8'
)
cursor = cafe24_db.cursor(pymysql.cursors.DictCursor)

sql = 'show tables;'
cursor.execute(sql)
tables = cursor.fetchall()

# 마지막으로 수집한 데이터 확인

In [13]:
for table in tables:
    table = table['Tables_in_statics']
    if table.startswith('view') or  table.startswith('fbig_'):
        continue
    sql = 'select max(date) as enddate, min(date) as begindate from {}'.format(table)
    cursor.execute(sql)
    tmp = cursor.fetchone()
    enddate = tmp['enddate']
    begindate = tmp['begindate']
    
    print( enddate, begindate, table)


2020-12-15 2020-02-12 day_category
2020-12-15 2019-01-01 day_category_product
2020-12-15 2019-01-01 day_domain
2020-12-15 2019-01-01 day_new_member
2020-10-20 21:32:20 2019-01-01 00:36:43 day_order
2020-12-15 2019-01-01 day_page_depth
2020-12-15 2019-01-01 day_pay_step
2020-12-15 2019-01-01 day_product
2020-12-15 2019-01-01 day_product_option
2020-12-15 2019-01-01 day_revisit_period
2020-12-15 2019-01-01 day_sales
2020-12-15 2019-01-01 day_search_engine
2020-12-15 2019-01-01 day_search_keyword
2020-12-15 2019-01-01 day_url
2020-12-15 2019-01-01 day_visit_count
2020-12-15 2019-01-01 hour_first_re_count_amout
2020-12-15 2019-01-01 hour_payment
2020-12-15 2019-01-01 hour_product
2020-12-15 2019-01-01 hour_view_visit
2020-11-16 2019-08-13 shopify_day_product


# 수집할 기간 설정
수집데이터 저장 경로 설정

In [16]:
today = datetime.datetime.today()
date_li = get_date_range( datetime.datetime(2020,10, 22), datetime.datetime(2020, 10, 21) )
maxdate = max(date_li).strftime('%Y%m%d')
mindate = min(date_li).strftime('%Y%m%d')

down_path = '/Users/dooyeoung/Documents/dashboard/download_{}_{}'.format(maxdate, mindate)

if os.path.exists(down_path) == False:
    os.mkdir(down_path)

[datetime.datetime(2020, 10, 22, 0, 0)]

# cafe24 로그인후 접속통계로 이동

In [8]:
options = webdriver.ChromeOptions()

hidden_browser = False
prefs = {'download.default_directory' : down_path}
options.add_experimental_option('prefs', prefs)
if hidden_browser:
    options.add_argument('headless')
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")
driver = webdriver.Chrome(executable_path='/Users/dooyeoung/Documents/projects/webdriver/chromedriver', 
                          chrome_options=options)



mallid = ''
passwd = ''
#  카페24 로그인
driver.get('https://eclogin.cafe24.com/Shop/')
driver.execute_script('''
document.getElementById('mall_id').value = {};
document.getElementById('userpasswd').value = '{}';
'''.format(mallid, passwd))
driver.find_element_by_css_selector('#tabAdmin > div > fieldset > p.gButton > a').click()
sleep(2)


# 비밀번호 변경하기 있는경우
sleep(3)
pwchange = driver.find_elements_by_css_selector('#iptBtnEm')
if len(pwchange) > 0:
    pwchange[0].click()
    sleep(1)
    
    
driver.get('https://'+mallid+'.cafe24.com/disp/admin/shop1/shop/frame?menu=270')
main_handle = driver.current_window_handle

driver.find_element_by_css_selector('#QA_Lnb_Menu1640').click()
driver.switch_to_window(driver.window_handles[-1])
report_handle = driver.current_window_handle

In [10]:
def change_date(date):
    driver.find_element_by_css_selector("a[onclick^='datePicker']").click()
    sleep(0.1)
    driver.execute_script("javascript:pickTime_1('{}', '{}', '{}');".format(date.year, date.month, date.day));
    sleep(0.1)
    driver.execute_script("javascript:pickTime_2('{}', '{}', '{}');".format(date.year, date.month, date.day));
    sleep(0.1)
    driver.find_element_by_css_selector("a[onclick^='submit_form_period']").click()
    sleep(0.5)
    
def change_date_range(begin_date, end_date):
    driver.find_element_by_css_selector("a[onclick^='datePicker']").click()
    sleep(0.1)
    driver.execute_script("javascript:pickTime_1('{}', '{}', '{}');".format(begin_date.year, begin_date.month, begin_date.day));
    sleep(0.1)
    driver.execute_script("javascript:pickTime_2('{}', '{}', '{}');".format(end_date.year, end_date.month, end_date.day));
    sleep(0.1)
    driver.find_element_by_css_selector("a[onclick^='submit_form_period']").click()
    sleep(0.5)
    
def parse_table():
    columns = [th.text.replace(' ','') for th in driver.find_elements_by_css_selector('#tbl_data thead th')]
    innerHTML = driver.find_element_by_css_selector('#tbl_data tbody:nth-child(2)').get_attribute('innerHTML')
    soup = BeautifulSoup(innerHTML, 'lxml')
    
    if len(innerHTML.strip()) == 0:
        tmp = driver.find_elements_by_css_selector('#detail_pfm_total')
        if len(tmp) > 0:
            innerHTML = driver.find_element_by_css_selector('#detail_pfm_total').get_attribute('innerHTML')
            soup = BeautifulSoup(innerHTML, 'lxml')
    
    
    tr_li = soup.select('tr')
    row_li = []
    row = None
    for tr in tr_li :
        row = [td.text for td in tr.select('td')]
        row_li.append(row)
    
    if row:
        if len(row) != len(columns):
            columns = None
    
    return pd.DataFrame(row_li, columns= columns)

def scrap(url, date_li, gather_sub= True, sub_data_columns=None, time="time" ):
    '''
    '''
    # 전체 페이지수 접속
    #url = 'https://elg-web.cafe24.com/elg/rpt/rpt_con_pvi_tab.php'
    page_name = url.split('/')[-1].split('.')[0]
    driver.get(url)
    
    sleep(0.2)
    page_size = driver.find_elements_by_css_selector('select[name="page_size"]')
    if len(page_size) > 0:
        try:
            driver.find_element_by_css_selector('select[name="page_size"] option[value="200"]').click()
            sleep(0.2)
        except:
            pass
    
    sleep(0.2)
    unit_format = driver.find_elements_by_css_selector('#unit_format')
    if len(unit_format) > 0:
        try:
            driver.find_element_by_css_selector('select#unit_format option[value="{}"]'.format(time)).click()
            sleep(0.2)
        except:
            pass
        
        
    scrap_data_li = []
    for date in date_li: 
        change_date(date) 
        scrap_data = parse_table() 
        scrap_data['date'] = date.strftime('%Y-%m-%d')
        scrap_data_li.append(scrap_data) 
        
        
        if sub_data_columns and gather_sub:
            down_sub_data(page_name, date, sub_data_columns)    

    if len(scrap_data_li)> 0:
        rdf = pd.concat(scrap_data_li).reset_index(drop=True)
        fname = '{}/{}.csv'.format(down_path, page_name)
        rdf.to_csv(fname, index=False)
        print('save {}'.format(fname))
    else:
        print('{} has no data'.format(fname) )

        
        
def scrap_onlyday(url, date_li, gather_sub= True, sub_data_columns=None, ):
    '''
    '''
    # 전체 페이지수 접속
    #url = 'https://elg-web.cafe24.com/elg/rpt/rpt_con_pvi_tab.php'
    page_name = url.split('/')[-1].split('.')[0]
    driver.get(url)
    
    sleep(0.2)
    page_size = driver.find_elements_by_css_selector('select[name="page_size"]')
    if len(page_size) > 0:
        try:
            driver.find_element_by_css_selector('select[name="page_size"] option[value="200"]').click()
            sleep(0.2)
        except:
            pass
    
    sleep(0.2)
    unit_format = driver.find_elements_by_css_selector('#unit_format')
    if len(unit_format) > 0:
        try:
            driver.find_element_by_css_selector('select#unit_format option[value="day"]').click()
            sleep(0.2)
        except:
            pass
        
        
    scrap_data_li = []
    change_date_range(min(date_li), max(date_li))
    
    scrap_data = parse_table() 
    #scrap_data['date'] = date.strftime('%Y-%m-%d')  

    if len(scrap_data)> 0:
        rdf = scrap_data
        fname = '{}/{}.csv'.format(down_path, page_name)
        rdf.to_csv(fname, index=False)
        print('save {}'.format(fname))
    else:
        print('{} has no data'.format(fname) )
    
def down_sub_data(page_name, date, columns):
    dtr_li = driver.find_elements_by_css_selector('#tbl_data > tbody:nth-child(2) > tr')

    tmp_df_li = []
    for tr in dtr_li :
        hour = tr.find_element_by_css_selector('td:nth-child(1)').text
        v1 = int(tr.find_element_by_css_selector('td:nth-child(3)').text.replace(',',''))
        v2 = int(tr.find_element_by_css_selector('td:nth-child(4)').text.replace(',',''))

        if v1+v2 > 0:
            tr.find_element_by_css_selector('td.td_align_c a').click()

            driver.switch_to_window(driver.window_handles[-1])
            sleep(0.5)
            pdtr_li = driver.find_elements_by_css_selector('table#tbl_data tbody tr')

            row_li = []
            for tr in pdtr_li:
                td_li = [td.text for td in tr.find_elements_by_css_selector('td')] 
                if len(td_li) > 0:
                    row_li.append(td_li)

            try:
                tmp = pd.DataFrame(row_li, columns=columns) 
                tmp = tmp[tmp[columns[0]] != '']
                tmp['date'] = date.strftime('%Y-%m-%d')
                tmp['hour'] = hour
                tmp_df_li.append(tmp)

                driver.close()
                driver.switch_to_window(report_handle)
            except Exception as e:
                print(e)
                display(pd.DataFrame(row_li))
                break
    
    if len(tmp_df_li) > 0:
    
        subrdf = pd.concat(tmp_df_li).reset_index(drop=True)
        fname = '{}/{}-{}.csv'.format(down_path,page_name, date.strftime('%Y%m%d'))
        subrdf.to_csv(fname, index=False)
        print('save {}'.format(fname))
    else:
        print('{} has no data'.format(date.strftime('%Y%m%d')))

# 접속 통계의 데이터 수집

In [11]:
scrap_onlyday('https://elg-web.cafe24.com/elg/rpt/rpt_con_vis_tab.php', date_li)
scrap_onlyday('https://elg-web.cafe24.com/elg/rpt/rpt_con_uni_tab.php', date_li)
scrap_onlyday('https://elg-web.cafe24.com/elg/rpt/rpt_con_fir_tab.php', date_li)
scrap_onlyday('https://elg-web.cafe24.com/elg/rpt/rpt_con_pvi_tab.php', date_li)

scrap('https://elg-web.cafe24.com/elg/rpt/rpt_sal_pmd_tab.php', date_li,) # 결제액
scrap('https://elg-web.cafe24.com/elg/rpt/rpt_sal_stp_tab.php', date_li,) # 구매단계
scrap('https://elg-web.cafe24.com/elg/rpt/rpt_cts_pag_tab.php', date_li,) # 페이지 체류시간 
scrap('https://elg-web.cafe24.com/elg/rpt/rpt_buy_rec_tab.php', date_li,) # 첫방문 재방문
scrap('https://elg-web.cafe24.com/elg/rpt/rpt_buy_vis_tab.php', date_li,) # 방문자 구매자

scrap('https://elg-web.cafe24.com/elg/rpt/rpt_pth_kdd_tab.php', date_li) # 검색어
scrap('https://elg-web.cafe24.com/elg/rpt/rpt_pth_dod_tab.php', date_li) # 방문 도메인
scrap('https://elg-web.cafe24.com/elg/rpt/rpt_pth_ufl_tab.php', date_li) # 상세 url
scrap('https://elg-web.cafe24.com/elg/rpt/rpt_pag_fad_tab.php', date_li) # 카테고리
scrap('https://elg-web.cafe24.com/elg/rpt/rpt_prd_sal_tab.php', date_li) # 상품 판매량
scrap('https://elg-web.cafe24.com/elg/rpt/rpt_bsk_pin_tab.php', date_li) # 장바구니 담기
scrap('https://elg-web.cafe24.com/elg/rpt/rpt_bsk_out_tab.php', date_li) # 장바구니 포기

scrap('https://elg-web.cafe24.com/elg/rpt/rpt_vis_rec_tab.php', date_li) # 다시찾아온 방문자
scrap('https://elg-web.cafe24.com/elg/rpt/rpt_vis_con_tab.php', date_li) # 방문 횟수별
scrap('https://elg-web.cafe24.com/elg/rpt/rpt_cts_sit_tab.php', date_li) # 일간 체류시간
scrap('https://elg-web.cafe24.com/elg/rpt/rpt_cts_ise_tab.php', date_li) # 내부 검색어
scrap('https://elg-web.cafe24.com/elg/rpt/rpt_buy_dur_tab.php', date_li) # 구매까지 걸린시간
scrap('https://elg-web.cafe24.com/elg/rpt/rpt_vis_nmb_tab.php', date_li) # 신규 회원수
scrap('https://elg-web.cafe24.com/elg/rpt/rpt_rou_dth_tab.php', date_li) # 방문경로 길이

scrap('https://elg-web.cafe24.com/elg/rpt/rpt_buy_res_tab.php', date_li, gather_sub=True, 
      sub_data_columns=['상품명','처음구매','재구매','재구매율','처음구매','재구매','재구매율']) # 방문경로 길이

scrap('https://elg-web.cafe24.com/elg/rpt/rpt_prd_cas_tab.php', date_li, gather_sub=True,
      sub_data_columns=['상품명','노출수','담긴수','판매건수','판매물품수','판매율','판매금액']) # 방문경로 길이

scrap('https://elg-web.cafe24.com/elg/rpt/rpt_pth_egd_tab.php', date_li, gather_sub=True,
     sub_data_columns=['검색어','방문수','구매건수','구매율','매출액','방문당매출','구매당매출']) # 검색엔진


scrap('https://elg-web.cafe24.com/elg/rpt/rpt_pag_fad_tab.php', date_li) # 카테고리
scrap('https://elg-web.cafe24.com/elg/rpt/rpt_prd_cas_tab.php', date_li, gather_sub=True,
      sub_data_columns=['상품명','노출수','담긴수','판매건수','판매물품수','판매율','판매금액']) # 방문경로 길이

save /Users/dooyeoung/Documents/dashboard/download_20201022_20201022/rpt_con_vis_tab.csv
save /Users/dooyeoung/Documents/dashboard/download_20201022_20201022/rpt_con_uni_tab.csv
save /Users/dooyeoung/Documents/dashboard/download_20201022_20201022/rpt_con_fir_tab.csv
save /Users/dooyeoung/Documents/dashboard/download_20201022_20201022/rpt_con_pvi_tab.csv
save /Users/dooyeoung/Documents/dashboard/download_20201022_20201022/rpt_sal_pmd_tab.csv
save /Users/dooyeoung/Documents/dashboard/download_20201022_20201022/rpt_sal_stp_tab.csv
save /Users/dooyeoung/Documents/dashboard/download_20201022_20201022/rpt_cts_pag_tab.csv
save /Users/dooyeoung/Documents/dashboard/download_20201022_20201022/rpt_buy_rec_tab.csv
save /Users/dooyeoung/Documents/dashboard/download_20201022_20201022/rpt_buy_vis_tab.csv
save /Users/dooyeoung/Documents/dashboard/download_20201022_20201022/rpt_pth_kdd_tab.csv
save /Users/dooyeoung/Documents/dashboard/download_20201022_20201022/rpt_pth_dod_tab.csv
save /Users/dooyeoung

# 일별 매출 데이터 수집

In [12]:
driver.close()
driver.switch_to_window(main_handle)
tmp_li = []
for date in date_li:
    date = date.strftime('%Y-%m-%d')
    url = 'https://'+mallid+'.cafe24.com/disp/admin/shop1/report/ProductPrdchart?start_date={}&end_date={}&searchDateRange=7&sEnableCategory=&rows=100&eProductSearchType=product_name&eOrderProductText=&eOrderProductNo=&iStartProductPrice=0&iEndProductPrice=0&sManufacturerCode=A&sSupplierCode=A&sBrandCode=A&sTrendCode=A&sClassificationCode=A&sMobileFlag=ALL&sOverseaFlag=ALL&sOrderBy=sale_cnt&sType=item&sCategory-1=0&sCategory-2=0&sCategory-3=0&sCategory-4=0&page={}'
    driver.get(url.format(date, date, 1))
    page_li = driver.find_elements_by_css_selector('.mPaginate li')
    
    row_li = []
    
    for page in range(1, len(page_li)+1):
        if page == 1:
            pass
        else:
            driver.get(url.format(date, date, page))
        
    
        tbody = driver.find_element_by_css_selector('.mBoard.typeList tbody')
    
        soup = BeautifulSoup(tbody.get_attribute('innerHTML'), 'lxml')
        tr_li = soup.select('tr')
    
        for tr in tr_li:
            try:
                prdprice = int(tr.select_one('td:nth-of-type(4)').getText().replace(',',''))
                prdqty = int(tr.select_one('td:nth-of-type(8)').getText())
                prdname = tr.select_one('td:nth-of-type(3) p').getText()
                prdname = prdname.replace('\n','').strip()
                prdsum = int(tr.select_one('td:nth-of-type(9)').getText().replace(',',''))

                option = ''
                options = tr.select('td:nth-of-type(3) ul.etc > li')
                if len(options) > 0: 
                    option = options[0].getText().split(':')[-1]
                    option = re.sub('[0-9]{1,2}.', '',option).strip()

                row = {'상품명':prdname, '옵션':option, '판매수량':prdqty,'판매가':prdprice, '판매합계':prdsum}
                row_li.append(row)
            except:
                print('has not data {}'.format(date))
                pass
    
    tmp = pd.DataFrame(row_li)
    tmp['date'] = date
    tmp_li.append(tmp)
pd.concat(tmp_li).to_csv('{}/day_product_option.csv'.format(down_path),index=False)


In [13]:
driver.quit()